#  Project Description

##  Background

Eye tracking is a technology that is used to measure the movement and position of the eye. Eye tracking can be used to obtain a variety of information, such as where someone is looking (also known as the gaze point). The raw eye tracking data cann also be used to engineer new features - eye tracking events - which can further be used to obtain more information. 

The types of eye tracking events that we can measure for include fixations, which are periods of time where the eye fixates on a target. There are saccades where the eyes move between points of fixations. There are also post-saccidic oscillations and glissades where the eye will oscillate after a saccade before settling to a fixation point. Post-saccadic oscillations overshoot the target, while glissades undershoot.

These types of events can be measured by applying different threshold techniques. I-VT applies a velocity threshold; If the speed between two gaze points is below a certain threshold, it is identified as a fixation. If the speed is above the threshold, it is a  saccade. There is also a dispersion/distance based method as well known as I-DT, that uses the distance between the gaze points instead to classify either fixations and saccades. These threshold algorithms are common in practice, but do not have the ability to classify more complex events. 

For the purpose of performing the I-VT  algorithm, a speed of 0.5px/ms was selected, and a dispersion of 1º was selected for I-DT.

## Dataset

For the following notebook, the dataset used is from a study performed in the University of Guelph DRiVE lab. Particpants wore eye-tracking glasses (Tobii Pro 3 glasses) and drove an OKTAL driving simulator. The dataset contains 72 participants that are randomly separated into train, test and validation sets. This will prevent leakage amonst the different particpant data.  Each of the files contains 3 different sets of data. There is some device information that is read in and in the sheet titled 'Event Data'. There is IMU sensor data in the sheet titled 'IMU Data'. The eye tracking data is in the sheet titled 'Gaze Data'. The sheets have 4, 22 and 11 columns respectively. The data  from the eye tracker is collected at 60Hz, and each participant file has roughly 20000 records in each file. The data is pre-split to ensure that there is no leakage between participant data, which could affect the training of the models, and to ensure a more consistent evaluation of the performance of the models.

## Procedure

1. The Gaze Data is read into the notebook using an Excel library.
2. For each participant file, any gaps in the data are filled in using linear interpolation first. 
3. Next, every two records are taken to calculate the labels using I-VT and I-DT and these are stored into a new dataframe. 

# Set Up Python Notebook

## Import Python Libraries

In [13]:
import os
from os import listdir
import pandas as pd

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

# import spark libraries
import findspark
findspark.init()
from pyspark.sql import SparkSession

import pyspark.pandas as ps
from pyspark.sql.functions import col
from pyspark.sql.functions import when
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql import functions as F

## Define Global Variables

In [14]:
datasets = ['dataset_training','dataset_testing','dataset_validation'] # directories for training, testing and valdiation
sheet = 'Gaze Data' # name of sheet with the eye tracking data

column_names = ['Type', 'Timestamp', 'Data_Gaze2D_X', 'Data_Gaze2D_Y', 'Data_Gaze3D_X',
       'Data_Gaze3D_Y', 'Data_Gaze3D_Z', 'Data_Eyeleft_Gazeorigin_X',
       'Data_Eyeleft_Gazeorigin_Y', 'Data_Eyeleft_Gazeorigin_Z',
       'Data_Eyeleft_Gazedirection_X', 'Data_Eyeleft_Gazedirection_Y',
       'Data_Eyeleft_Gazedirection_Z', 'Data_Eyeleft_Pupildiameter',
       'Data_Eyeright_Gazeorigin_X', 'Data_Eyeright_Gazeorigin_Y',
       'Data_Eyeright_Gazeorigin_Z', 'Data_Eyeright_Gazedirection_X',
       'Data_Eyeright_Gazedirection_Y', 'Data_Eyeright_Gazedirection_Z',
       'Data_Eyeright_Pupildiameter']

# createempty dataframe variables
df_train = None
df_test = None
df_validation = None

## Create Spark Session

In [15]:
spark = SparkSession.builder\
    .appName("Cis6180_FinalProject")\
    .config("spark.driver.memory", "48g")\
    .config("spark.memory.offHeap.enabled","true")\
    .config("spark.memory.offHeap.size","10g")\
    .config("spark.executor.memory", "48g")\
    .config("spark.executor.cores", 4)\
    .config("spark.shuffle.service.enabled", True)\
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.minExecutors", 1)\
    .config("spark.dynamicAllocation.maxExecutors", 4)\
    .getOrCreate()


## Create List of Files to Import

In [16]:
file_list = []

# iterate through all the files in the dataset to ge
for ds_num,dataset in enumerate(datasets):
    data_files = listdir(dataset)
    for f_num,f in enumerate(data_files):
        file_path = dataset + '/' + f # file path is the relative file path for the current excel file
        file_list.append((file_path,ds_num))

print(len(file_list))

73


## Iterate Through All Files to Read into DataFrames

In [17]:
file_num = 1

# iterate through the list of files to read them, interpolate missing data, merge them and convert to pyspark
# this takes 5-20 minutes and should be optimized in the future

for file_path in file_list:
    print(f'File: {file_num}/{len(file_list)} {file_path[0]}')
    
    # read the dataframe as python and then convert to pyspark since it has to be read in from excel spreadsheet
    ppdf = pd.read_excel(io=file_path[0],sheet_name=sheet) # read excel as pandas
    ppdf.columns = column_names
    ppdf = ppdf.drop(['Type'],axis=1)
    ppdf = ppdf.interpolate(method='linear',limit_direction='both')

    ppdf_first = ppdf.iloc[:-1] # create copy of ppdf and remove last row
    ppdf_second = ppdf.copy().iloc[1:] # create a copy of ppdf and remove the first row
    ppdf_second.reset_index(drop=True, inplace=True)  # Reset index to ensure alignment

    ppdf_second.columns = [col + '_2' for col in ppdf_second.columns]# rename the columns of the second copy
    ppdf_combined = pd.concat([ppdf_first,ppdf_second],axis=1) # merge first copy and second copy side by side

    psdf = ps.from_pandas(ppdf_combined)
    
    if file_path[1] == 0:  # train
        if df_train is None:
            df_train = psdf
        else:
            df_train.append(psdf, ignore_index = True)# append current dataset to the one that is already existing

    elif file_path[1] == 1:  # test
        if df_test is None:
            df_test = psdf
        else:
            df_test.append(psdf, ignore_index = True)# append current dataset to the one that is already existing
    elif file_path[1] == 2:  # validation
        if df_validation is None:
            df_validation = psdf
        else:
            df_validation.append(psdf, ignore_index = True)

    file_num+=1

File: 1/73 dataset_training/eye-data-10327.xlsx
File: 2/73 dataset_training/eye-data-12471.xlsx


c:\Users\01eco\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\pandas\frame.py:8890: FutureWarning: The DataFrame.append method is deprecated and will be removed in 4.0.0. Use pyspark.pandas.concat instead.
  warnings.warn(


File: 3/73 dataset_training/eye-data-18514.xlsx


c:\Users\01eco\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\pandas\frame.py:8890: FutureWarning: The DataFrame.append method is deprecated and will be removed in 4.0.0. Use pyspark.pandas.concat instead.
  warnings.warn(


File: 4/73 dataset_training/eye-data-20116.xlsx


...

c:\Users\01eco\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\pandas\frame.py:8890: FutureWarning: The DataFrame.append method is deprecated and will be removed in 4.0.0. Use pyspark.pandas.concat instead.
  warnings.warn(


File: 72/73 dataset_validation/eye-data-51553.xlsx


c:\Users\01eco\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\pandas\frame.py:8890: FutureWarning: The DataFrame.append method is deprecated and will be removed in 4.0.0. Use pyspark.pandas.concat instead.
  warnings.warn(


File: 73/73 dataset_validation/eye-data-64087.xlsx


c:\Users\01eco\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\pandas\frame.py:8890: FutureWarning: The DataFrame.append method is deprecated and will be removed in 4.0.0. Use pyspark.pandas.concat instead.
  warnings.warn(


In [18]:
df_train.dtypes

Timestamp                          float64
Data_Gaze2D_X                      float64
Data_Gaze2D_Y                      float64
Data_Gaze3D_X                      float64
Data_Gaze3D_Y                      float64
Data_Gaze3D_Z                      float64
Data_Eyeleft_Gazeorigin_X          float64
Data_Eyeleft_Gazeorigin_Y          float64
Data_Eyeleft_Gazeorigin_Z          float64
Data_Eyeleft_Gazedirection_X       float64
Data_Eyeleft_Gazedirection_Y       float64
Data_Eyeleft_Gazedirection_Z       float64
Data_Eyeleft_Pupildiameter         float64
Data_Eyeright_Gazeorigin_X         float64
Data_Eyeright_Gazeorigin_Y         float64
Data_Eyeright_Gazeorigin_Z         float64
Data_Eyeright_Gazedirection_X      float64
Data_Eyeright_Gazedirection_Y      float64
Data_Eyeright_Gazedirection_Z      float64
Data_Eyeright_Pupildiameter        float64
Timestamp_2                        float64
Data_Gaze2D_X_2                    float64
Data_Gaze2D_Y_2                    float64
Data_Gaze3D

In [19]:
df_train.columns

Index(['Timestamp', 'Data_Gaze2D_X', 'Data_Gaze2D_Y', 'Data_Gaze3D_X',
       'Data_Gaze3D_Y', 'Data_Gaze3D_Z', 'Data_Eyeleft_Gazeorigin_X',
       'Data_Eyeleft_Gazeorigin_Y', 'Data_Eyeleft_Gazeorigin_Z',
       'Data_Eyeleft_Gazedirection_X', 'Data_Eyeleft_Gazedirection_Y',
       'Data_Eyeleft_Gazedirection_Z', 'Data_Eyeleft_Pupildiameter',
       'Data_Eyeright_Gazeorigin_X', 'Data_Eyeright_Gazeorigin_Y',
       'Data_Eyeright_Gazeorigin_Z', 'Data_Eyeright_Gazedirection_X',
       'Data_Eyeright_Gazedirection_Y', 'Data_Eyeright_Gazedirection_Z',
       'Data_Eyeright_Pupildiameter', 'Timestamp_2', 'Data_Gaze2D_X_2',
       'Data_Gaze2D_Y_2', 'Data_Gaze3D_X_2', 'Data_Gaze3D_Y_2',
       'Data_Gaze3D_Z_2', 'Data_Eyeleft_Gazeorigin_X_2',
       'Data_Eyeleft_Gazeorigin_Y_2', 'Data_Eyeleft_Gazeorigin_Z_2',
       'Data_Eyeleft_Gazedirection_X_2', 'Data_Eyeleft_Gazedirection_Y_2',
       'Data_Eyeleft_Gazedirection_Z_2', 'Data_Eyeleft_Pupildiameter_2',
       'Data_Eyeright_Gazeorigin_X

In [21]:
df_train.count()

Py4JJavaError: An error occurred while calling o33182.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 7.0 failed 1 times, most recent failure: Lost task 2.0 in stage 7.0 (TID 21) (ER1CHS-LAPT0P executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	a